# 3.0 Full MNIST Test

In [1]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)
df = {}

import clustergrammer_groupby as cby
import gene_exp_10x

In [2]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
%%time
net.load_file('../data/big_data/MNIST_row_labels.txt')
df['ini'] = net.export_df()

CPU times: user 50.3 s, sys: 1.38 s, total: 51.6 s
Wall time: 51.8 s


In [4]:
df['ini'].shape

(784, 70000)

In [5]:
df['ini'].head()

,Zero-0,Zero-1,Zero-2,Zero-3,Zero-4,Zero-5,Zero-6,Zero-7,Zero-8,Zero-9,...,Nine-6948,Nine-6949,Nine-6950,Nine-6951,Nine-6952,Nine-6953,Nine-6954,Nine-6955,Nine-6956,Nine-6957
pos_0-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Add categories

In [6]:
cols = df['ini'].columns.tolist()
new_cols = []
for inst_col in cols:
    new_col = (inst_col, 'Digit: ' + inst_col.split('-')[0])
    new_cols.append(new_col)

df['digits'] = deepcopy(df['ini'])
df['digits'].columns = new_cols

In [7]:
df['digits'].head()

,"(Zero-0, Digit: Zero)","(Zero-1, Digit: Zero)","(Zero-2, Digit: Zero)","(Zero-3, Digit: Zero)","(Zero-4, Digit: Zero)","(Zero-5, Digit: Zero)","(Zero-6, Digit: Zero)","(Zero-7, Digit: Zero)","(Zero-8, Digit: Zero)","(Zero-9, Digit: Zero)",...,"(Nine-6948, Digit: Nine)","(Nine-6949, Digit: Nine)","(Nine-6950, Digit: Nine)","(Nine-6951, Digit: Nine)","(Nine-6952, Digit: Nine)","(Nine-6953, Digit: Nine)","(Nine-6954, Digit: Nine)","(Nine-6955, Digit: Nine)","(Nine-6956, Digit: Nine)","(Nine-6957, Digit: Nine)"
pos_0-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Make signatures and Predict Categories

In [8]:
%%time
pval_cutoff = 0.001
df['cat_sig'], keep_genes, keep_genes_dict = cby.generate_signatures(df['digits'], 'Digit', 
                                                                     pval_cutoff=pval_cutoff)
print(df['cat_sig'].shape)

/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(678, 10)
CPU times: user 5.4 s, sys: 2.04 s, total: 7.44 s
Wall time: 7.47 s


### Predict categories

In [9]:
%%time
df['pred_cat'], df['sig_sim'], y_info = cby.predict_cats_from_sigs(df['digits'], df['cat_sig'], 
                                                                                  predict_level='Predict Digit')

CPU times: user 990 ms, sys: 498 ms, total: 1.49 s
Wall time: 1.38 s


In [10]:
%%time 
df['conf'], true_count, pred_count, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
real_fraction_correct = deepcopy(fraction_correct)
print(real_fraction_correct)

0.814828571429
CPU times: user 173 ms, sys: 4.69 ms, total: 177 ms
Wall time: 177 ms


### Run Time for Predicting 70,000 Digits
It takes about two seconds to predict 70,000 digits (using the full MNIST dataset). We can extrapolate the run time for larger datasets.

### Estimated Time for 1 Million Cells

In [11]:
print(2 * (1000000/70000)/60, 'minutes for 1 million datapoints')

0.4761904761904762 minutes for 1 million datapoints


In [12]:
df_list = []
for i in range(15):
    df_list.append(df['digits'])
df['large'] = pd.concat(df_list, axis=1)
df['large'].shape

(784, 1050000)

### ~30 seconds to predict 1,000,000 digits

In [13]:
%%time
df['pred_cat'], df['sig_sim'], y_info = cby.predict_cats_from_sigs(df['large'], df['cat_sig'], 
                                                                                  predict_level='Predict Digit')

CPU times: user 19.2 s, sys: 10.6 s, total: 29.8 s
Wall time: 28 s


In [14]:
%%time 
df['conf'], true_count, pred_count, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
real_fraction_correct = deepcopy(fraction_correct)

CPU times: user 2.55 s, sys: 48.6 ms, total: 2.6 s
Wall time: 2.6 s


In [15]:
print(real_fraction_correct)

0.814828571429
